# Black-Scholes Options Pricing calculator

Developed in 1973 by Fischer Black, Robert Merton, and Myron Scholes, the Black-Scholes model was the first widely used mathematical method to calculate the theoretical value of an option contract, using current stock prices, expected dividends, the option's strike price, expected interest rates, time to expiration, and expected volatility.

### Import dependencies

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

### User-defined Class and Functions

 The Black-Scholes model makes certain assumptions:

    No dividends are paid out during the life of the option.
    Markets are random, and cannot be predicted
    There are no transaction costs in buying the option.
    The risk-free rate and volatility of the underlying asset are known and constant.
    The returns of the underlying asset are normally distributed.
    The option is European and can only be exercised at expiration.

In [ ]:
# Initialise a Black-Scholes Merton Model class, adding the required parameters
''' 
    S = current stock price
    K = strike price
    r = risk-free interest rate
    T = time to maturity
    sigma = volatility
    q = divident yield
'''
class BSMModel:
    def __init__(self, S, K, r, T, sigma, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r 
        self.sigma = sigma
        self.q = q
        
    def d1(self):
        return (np.log(self.S/self.K) + (self.r -self.q + self.sigma**2/2)*self.T)/(self.sigma*np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma*np.sqrt(self.T)
    
    # Calculates call value
    def _call_value(self):
        return self.S*np.exp(-self.q*self.T)*norm.cdf(self.d1(),0,1) - self.K*np.exp(-self.r*self.T) * norm.cdf(self.d2(),0,1)

    # Calculates put value                
    def _put_value(self):
        return self.K*np.exp(-self.r*self.T)*norm.cdf(-self.d2(),0,1) - self.S*np.exp(-self.q*self.T)*norm.cdf(-self.d1(),0,1)
    
    # Calculates the call or put value of the option
    def price(self, type_ = "C"):
        try:
            if type_ == "C":
                return self._call_value()
            elif type_ == "P":
                return self._put_value() 
            elif type_ == "B":
                return  {"call": self._call_value(), "put": self._put_value()}
        except TypeError:
            print("Unrecognized type")
        return None

    # Option Greeks

    def delta(self, type_ = "C"):
        if type_ not in ["C", "P", "B"]:
            print("Unrecognized type")
            return None
    
        if type_ == "C":
            return self._call_value()
        elif type_ == "P":
            return self._put_value() 
        elif type_ == "B":
            return {"call": self._call_value(), "put": self._put_value()}

    def gamma(self):
        gamma = norm.pdf(self.d1(), 0, 1)/ (self.S * self.sigma * np.sqrt(self.T))
        return gamma
       
    def theta(self, type_ = "C"):
        if type_ not in ["C", "P"]:
            print("Unrecognized type")
            return None
        
        if type_ == "C":
            theta = - ((self.S * norm.pdf(self.d1(), 0, 1) * self.sigma) / (2 * np.sqrt(self.T))) - self.r * self.K * np.exp(-self.r*self.T) * norm.cdf(self.d2(), 0, 1)

        elif type_ == "P":
            theta = - ((self.S * norm.pdf(self.d1(), 0, 1) * self.sigma) / (2 * np.sqrt(self.T))) + self.r * self.K * np.exp(-self.r*self.T) * norm.cdf(-self.d2(), 0, 1)
        return theta/365
      
    def vega(self):
        vega = self.S * np.sqrt(T) * norm.pdf(self.d1(), 0, 1) * 0.01
        return vega

    def rho(self, type_ = "C"):
        if type_ not in ["C", "P"]:
            print("Unrecognized type")
            return None
        if type_ == "C":
            rho = 0.01 * self.K * self.T * np.exp(-self.r*self.T) * norm.cdf(self.d2(), 0, 1)
        elif type_ == "P":
            rho = 0.01 * -self.K * self.T * np.exp(-self.r*self.T) * norm.cdf(-self.d2(), 0, 1)
        return rho

    def __str__(self):
        return f"BSMModel(S={self.S}, K={self.K}, r={self.r}, T={self.T}, sigma={self.sigma}, q={self.q})"
    

### User defined global functions

The function below generates a geometric random walk (monte carlo simulation)  
The value that this converges to should converge to the Black-Scholes model value as the N becomes arbitrarily large

In [ ]:
def geo_mc_randomwalk(S, T, r, q, sigma, steps, N):
    """
    #S = Current stock Price
    #K = Strike Price
    #T = Time to maturity in years
    #r = risk free interest rate
    #q = dividend yield
    # sigma = volatility 
    """
    dt = T/steps
    
    ST = np.log(S) + np.cumsum(((r - q - sigma**2/2)*dt + sigma*np.sqrt(dt) * np.random.normal(size=(steps,N))),axis=0)

    return np.exp(ST)

### Main Function

In [ ]:
if __name__ == "__main__":
    # Set values for S,K,r,T,q,sigma to test the model
    S = 100
    K = 110
    r = 0.05
    T = 0.5
    q = 0
    sigma = 0.25

    steps = 100
    N1 = 1000
    N2 = 100000
    
    # Test the model with the prior declared values
    test_model = BSMModel(S, K, r, T, sigma)

    print(test_model)
    print(test_model.price(type_ = "B"))
    print(f'delta is: {test_model.delta()}')
    print(f'gamma is: {test_model.gamma()}')
    print(f'theta is: {test_model.theta()}')
    print(f'vega is: {test_model.vega()}')
    print(f'rho is: {test_model.rho()}')

    paths1 = geo_mc_randomwalk(S,T,r, q,sigma,steps,N1)
    paths2 = geo_mc_randomwalk(S,T,r, q,sigma,steps,N2)
    
    plt.plot(paths1, color='k')  # Set line color to black for grayscale
    plt.xlabel("Time Increments")
    plt.ylabel("Stock Price")
    plt.title("Geometric Brownian Motion")
    plt.show()

    BSM_price = test_model._call_value()
    payoffs1 = np.maximum(paths1[-1]-K, 0)
    option_price1 = np.mean(payoffs1)*np.exp(-r*T)
    epsilon1 = np.abs(BSM_price - option_price1)

    payoffs2 = np.maximum(paths2[-1]-K, 0)
    option_price2 = np.mean(payoffs2)*np.exp(-r*T)
    epsilon2 = np.abs(BSM_price - option_price2)

    print(f"Black Scholes Call Price is {BSM_price}")
    print(f"Simulated markov chain price with 1000 steps is {option_price1}")
    print(f"Simulated markov chain price with 10 000 steps is {option_price2}")
    print(f"\nThe difference between between BSM call price and simulation is: {epsilon1} for 1000 step")
    print(f"The difference between between BSM call price and simulation is: {epsilon2} for 10 000 step")